## Playing with our Sample Data

At this point we have a sample of ten active bikes saved locally in `csv` format as `sample_trips.csv`. We need to grab that file and throw it through the wringer to format it into a GeoJSON store that can be consumed by `d3.js`. What wringer? Keep reading...

In [1]:
import googlemaps
import json
import os
from polyline.codec import PolylineCodec
import geojson
import pandas as pd
import numpy as np
import random
import mplleaflet
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
sample_trips = pd.read_csv("../data/part_1/sample_trips.csv", index_col=0)

In [3]:
sample_trips['starttime'] = sample_trips['starttime'].map(lambda x:
                                                          np.datetime64(datetime.datetime.strptime(x, "%Y-%d-%m %H:%M:%S")))
sample_trips['stoptime'] = sample_trips['stoptime'].map(lambda x:
                                                        np.datetime64(datetime.datetime.strptime(x, "%Y-%d-%m %H:%M:%S")))

We now need pass the data through the Google Maps API and passing it to a `GeoJSON`-formatted `dict` that we can save and have `d3.js` consume. This is a complex process.

## Getting and Parsing Directions

### Directions via the Google Maps API

There's no way to know what path any of the bikers actually travelled. Maybe they had urgent business and needed to get there quickly, lest they miss a meeting. Or maybe they were just taking it on a tour of the town. We can make some attempt to classify trips as slow of fast based on their correspondance with average bike trips between the two stations in question, but knowing only the endpoints, that's the best we can.

So instead we will call the points into the [Google Maps Directions API](https://developers.google.com/maps/documentation/directions/). I use the `googlemaps` module to do the deed. This will give us a "best" path for this trip (note that Google Maps tends to favor protected bike lanes in the results it returns when it's in Biking Mode, for obvious reasons). 

The entire trip is presented not as a single continuous entity either, but as a set of so-called "legs". Take the highway, turn right onto the road, etc. etc. Each of these legs has a polygon of coordinates, a polygon that is compressed into and returned as something called an [encoded polyline](https://developers.google.com/maps/documentation/utilities/polylinealgorithm). So effectively we have to loop over each leg in the path, collect the polyline, decode it, and concatenate that set of coordinates to our working list.


### Side-note: Getting API Access

You need to have your Google Maps API credentials stored locally at `../credentials/google_maps_api_key.json` using the following format for the next few lines to work:

> `{ "key": "..." }`

See the [Google Developer Console](https://console.developers.google.com/) for information getting your own API key. Note that you will need a *browser key*, specifically, not the related but functionally different *server key*.

### Accounting for Rebalancing

Far from every trip in the CitiBike system stems from an actual bike ride. The trouble with bikeshare systems is that they are source-to-sink&mdash;since the original user does not have to take the bike back to where they started, the bikes tend to accumulate over time at whatever stations are most popular. Since those stations themselves have limited out-of-station bandwith as well as their own peculiar destination patterns, the CitiBike system effectively functions as a huge [Markov chain](https://en.wikipedia.org/wiki/Markov_chain).

This explains the phenomenon that every New Yorker has probably seen but many perhaps cannot place, the sensation that racks are empty at certain stations at some times and in some areas and completely stuffed at others. These kinds of things get mathematicians very excited ([very very excited](http://www.citylab.com/commute/2014/08/balancing-bike-share-stations-has-become-a-serious-scientific-endeavor/379188/)), but on a day-to-day level they mean that bikes tend to have to be rebalanced by so-called bike vans or whatever-ya-call-thems.

These trips do not show up as trips proper in the dataset. If you don't account for rebalancing trips you will be left wondering why your bike is teleporting around from time to time! They're fairly easily dispatched once you're aware of them, however.

In visualization terms my plan is to plot these trips exactly midway between the trips around them, with the simulated time taken dependent on the estimated time given by the Google Maps API. Note that this requires making a call to the Google Maps API in a different mode&mdash;the regular, car-based one, in this case, instead of the bike one we use for real trips.

Of course in reality the bikes tend to get ferried around all whily-nilly as all the errant bikes are collected for transfering (at least I assume, I actually have no idea how this stuff is done). But the whole thing is a simulation anyway so pipe down!

### Bike Trips, Rebalancing Trips

There are two cases under consideration here, **bike trips** and **rebalancing trips**.

In the case of bike trips we throw the start point and end point at the Google Maps API (in bike mode), extract the polyline, throw out the rest of the data, and embed the properties we have from the trips dataset into the GeoJSON `properties`.

In the case of a rebalancing trip, everything is a little more complicated. We can detect rebalancing trips by looking at reported trips in the dataset in which the bike seems to "warp" between one station and another. We make a request to the Google Maps API (in standard car mode this time) and extract the polyline again, but also extract the time estimate Google gives us for the trip. We measure the time between the two stations end and start times, respectively, and throw the rebalancing trip time exactly in between, shearing it off if it turns out to be longer than the actual time allowed (as is theoretically possible). We populate the properties from the two stations' data and this time data before finally saving the built-out string to GeoJSON.

In [4]:
def import_credentials(filename='google_maps_api_key.json'):
    path = '../credentials/{0}'.format(filename)
    if os.path.isfile(path):
        data = json.load(open(path))['key']
        return data
    else:
        raise IOError(
            'This API requires a Google Maps credentials token to work. Did you forget to define one?')
        
gmaps = googlemaps.Client(key=import_credentials())

First I wrote and tested a method which passes directions to Google Maps, and parses it down to coordinates, and a `mplleaflet` mapper to check the results.

### Bike Tripper

In [5]:
def get_bike_trip_path(start, end, client):
    req = client.directions(start, end, mode='bicycling')
    polylines = [step['polyline']['points'] for step in [leg['steps'] for leg in req[0]['legs']][0]]
    coords = []
    for polyline in polylines:
        coords += PolylineCodec().decode(polyline)
    return coords

In [6]:
def plot_path(coords):
    x_coords = [coord[1] for coord in coords] # Notice the position swap!
    y_coords = [coord[0] for coord in coords]
    plt.hold(True)
    plt.plot(x_coords, y_coords, 'b')
    # mplleaflet.display(fig=plt.figure())
    mplleaflet.show()

The following opens the full `Leaflet` plot in a seperate window.

In [7]:
test_path = get_bike_trip_path([40.76727216,-73.99392888], [40.701907,-74.013942], gmaps) # actual path from the data
plot_path(test_path)

In [8]:
import time
time.sleep(5)
# Otherwise the OS might move the file before mplleaflet finishes visualizing it.

try:
    os.remove("../figures/geocoder_test_path.html")
except OSError:
    pass
os.rename("_map.html", "../figures/geocoder_test_path.html") # Just to keep the directory tidy.

### Rebalancing Tripper

That verifies that our bike tripper is working. How about a rebalancing tripper?

This isn't the typical map function. Since we need context about the trip before and the trip after for this to work, we will need to iterate through the list of trips two at a time, spot differences in start and end points, and connect them up through the geocoder, requiring full GeoJSON output. In other words this isn't an operation that can be neatly vectorized!

In [9]:
def get_rebalancing_trip_path_time_estimate_tuple(start, end, client):
    req = client.directions(start, end, mode='driving')
    # return req
    # Get the time estimates.
    # Raw time estimate results are strings of the form "1 min", "5 mins", "1 hour 5 mins", "2 hours 5 mins", etc.
    time_estimates_raw = [step['duration']['text'] for step in [leg['steps'] for leg in req[0]['legs']][0]]
    time_estimate_mins = 0
    for time_estimate_raw in time_estimates_raw:
        # Can we really get an hour+ estimate biking within the city? Possibly not but I won't risk it.
        if "min" in time_estimate_raw and "hour" not in time_estimate_raw:
            time_estimate_mins += int(time_estimate_raw.split(" ")[0])
        elif "hour" in time_estimate_raw:
            time_estimate_mins += 60 * int(time_estimate_raw.split(" ")[0])
            if "min" in time_estimate_raw:
                time_estimate_mins += int(time_estimate_raw.split(" ")[2])
            else:
                # Uh-oh.
                pass
    # Get the polylines.
    polylines = [step['polyline']['points'] for step in [leg['steps'] for leg in req[0]['legs']][0]]
    coords = []
    for polyline in polylines:
        coords += PolylineCodec().decode(polyline)
    # Return
    return coords, time_estimate_mins

In [10]:
get_rebalancing_trip_path_time_estimate_tuple([40.76727216,-73.99392888], [40.701907,-74.013942], gmaps)

([(40.76723, -73.99396),
  (40.76713, -73.99372),
  (40.76713, -73.99372),
  (40.76651, -73.99418),
  (40.76621, -73.99442),
  (40.76605, -73.99453),
  (40.76589, -73.99466),
  (40.76526, -73.99511),
  (40.76497, -73.99532),
  (40.76463, -73.99557),
  (40.76401, -73.99602),
  (40.76373, -73.99622),
  (40.76339, -73.99647),
  (40.76277, -73.99693),
  (40.76256, -73.99708),
  (40.76243, -73.99717),
  (40.76213, -73.99739),
  (40.76152, -73.99785),
  (40.76085, -73.99832),
  (40.76016, -73.99882),
  (40.76011, -73.99885),
  (40.75992, -73.99897),
  (40.75953, -73.99924),
  (40.75953, -73.99924),
  (40.75906, -73.99957),
  (40.75891, -73.99968),
  (40.75868, -73.99984),
  (40.75831, -74.00014),
  (40.75768, -74.00059),
  (40.75707, -74.00106),
  (40.75677, -74.00128),
  (40.75646, -74.00152),
  (40.7562, -74.0017),
  (40.75607, -74.00178),
  (40.75594, -74.00188),
  (40.7558, -74.002),
  (40.75567, -74.0021),
  (40.75522, -74.00244),
  (40.75514, -74.00249),
  (40.75504, -74.00256),
  (40.

In [11]:
def get_list_of_rebalancing_frames_from_df(df):
    frames = []
    for a_minus_1, a in zip(range(len(df) - 1), range(1, len(df))):
        delta = df.iloc[[a_minus_1, a]]
        ind_1, ind_2 = delta.index.values
        if delta.ix[ind_1, 'end station id'] == delta.ix[ind_2, 'start station id']:
            continue
        else:
            frames.append(delta)
    return frames


def rebalanced(delta_df):
    ind_1, ind_2 = delta_df.index.values
    if delta_df.ix[ind_1, 'end station id'] == delta_df.ix[ind_2, 'start station id']:
        return False
    else:
        return True


def get_rebalancing_geojson_repr(delta_df):
    start_point = delta_df.iloc[0]
    end_point = delta_df.iloc[1]
    start_lat, start_long = start_point[["end station latitude", "end station longitude"]]
    end_lat, end_long = end_point[["start station latitude", "start station longitude"]]
    coords, time_estimate_mins = get_rebalancing_trip_path_time_estimate_tuple([40.76727216,-73.99392888],
                                                                               [40.701907,-74.013942], gmaps)
    midpoint_time = end_point['starttime'] + ((end_point['starttime'] - start_point['stoptime']) / 2)
    rebalancing_start_time = midpoint_time - datetime.timedelta(minutes=time_estimate_mins / 2)
    rebalancing_end_time = midpoint_time + datetime.timedelta(minutes=time_estimate_mins / 2)
    if rebalancing_start_time < start_point['stoptime']:
        rebalancing_start_time = start_point['stoptime']
    if rebalancing_end_time > end_point['starttime']:
        rebalancing_end_time = end_point['starttime']
    attributes = {
        "tripduration": time_estimate_mins * 60,
        "start station id": start_point['end station id'],
        "end station id": end_point['start station id'],
        "start station name": start_point['end station name'],
        "end station name": end_point['start station name'],
        "bikeid": start_point["bikeid"],
        "usertype": "Rebalancing",
        "birth year": 0.0,
        "gender": 3.0,
        "start station latitude": start_lat,
        "start station longitude": start_long,
        "end station latitude": end_lat,
        "end station longitude": end_long,
        "starttime": rebalancing_start_time.strftime("%Y-%d-%m %H:%M:%S"),
        "stoptime": rebalancing_end_time.strftime("%Y-%d-%m %H:%M:%S")
    }
    return geojson.Feature(geometry=geojson.LineString(coords, properties=attributes))

In [22]:
# NOTE: This *may* not work!
# sample_trips[sample_trips['bikeid'] == 17609].sort_values(by="starttime").head(2).to_csv(
#     "../data/part_1/non_rebalanced_sample.csv"
# )

# This definitely works.
with open("../data/part_1/non_rebalanced_sample.csv", "w") as f:
    f.write("""Index,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
287273,1315.0,2016-03-06 12:35:00,2016-03-06 12:56:56,358.0,Christopher St & Greenwich St,40.73291553,-74.00711384,530.0,11 Ave & W 59 St,40.771522,-73.99054100000002,17609.0,Subscriber,1981.0,1.0
719222,588.0,2016-03-06 14:38:55,2016-03-06 14:48:44,530.0,11 Ave & W 59 St,40.771522,-73.99054100000002,3163.0,Central Park West & W 68 St,40.7734066,-73.97782542,17609.0,Subscriber,1953.0,1.0""")

In [13]:
sample_trips.head(2).to_csv("../data/part_1/rebalanced_sample.csv")

## Format validation

Before we can go further, however, we have to address an issue that crops up. The Citibike dataset includes basic demographic information about bikes' users (`birth year` and `gender`) which is available only for subscribers (that is, records with `usertype=subscriber`). Non-subscribers (customers, probably primarily tourists, which are indicated by a `usertype=Customer` value) are not required to provide this data to ride, and so these columns are unavailable in these cases.

The Citibike dataset encodes unknown `gender` as 0.0 (`1.0` is male, `2.0` is female) and it encodes unknown `birth year` as a blank space which `pandas` converts to an `np.nan` value. However, the `geojson` spec doesn't support `np.nan` or other such sentinal values, so if we try to pass a `birth year=np.nan` to `geojson.dumps` and/or validate our data it will not take. This is easy to fix: to maintain formatting consistency we simply replace `birth year=np.nan` with `birth year=0.0` (as with `gender`) using `pd.fillna()`.


In [14]:
sample_trips = sample_trips.fillna(0.0)

## Make it a GeoJSON

The following method reads the list of paths associated with a particular bicycle from our dataset and packages it into a `GeoJSON` `FeatureCollection` containing the bicycle's entire trip-week. `GeoJSON` is a format for geographic data that `d3` understands (they were invented by the same guy, in fact, the one and only Mike Bostock).

The end result is going be a wrapper around a list of Features that look a lot like this:

    {"geometry": {"coordinates": [[40.72838, -73.98717], [...], ...]], "properties": {"bikeid": 20249.0, "birth year": 1983.0, "end station id": 268.0, "end station latitude": 40.71910537, "end station longitude": -73.99973337, "end station name": "Howard St & Centre St", "gender": 1.0, "start station id": 236.0, "start station latitude": 40.7284186, "start station longitude": -73.98713956, "start station name": "St Marks Pl & 2 Ave", "starttime": "2016-03-11 09:02:45", "stoptime": "2016-03-11 09:10:28", "tripduration": 462.0, "usertype": "Subscriber"}, "type": "LineString"}, "properties": {}, "type": "Feature"}

The difference between trip types is encoded in the `usertype` property. Bike trips have a `usertype` of `Subscriber` or `Customer`; rebalancing trips, by contrast, go under `rebalancing`.

In [15]:
def get_trips(df, bike_id, client):
    feature_list = []
    bike_df = df[df['bikeid'] == bike_id].sort_values(by='starttime')
    for a_minus_1, a in zip(range(len(bike_df) - 1), range(1, len(bike_df))):
        delta_df = bike_df.iloc[[a_minus_1, a]]
        ind_1, ind_2 = delta_df.index.values
#         print(ind_1, ind_2)
#         print(delta_df.index.values)
#         print(delta_df)
        start = delta_df.ix[ind_1]
        end = delta_df.ix[ind_2]
        path = get_bike_trip_path([start['start station latitude'], start['start station longitude']],
                                  [start['end station latitude'], start['end station longitude']],
                                  client)
        props = start.to_dict()
        props['starttime'] = props['starttime'].strftime("%Y-%d-%m %H:%M:%S")
        props['stoptime'] = props['stoptime'].strftime("%Y-%d-%m %H:%M:%S")
        feature_list.append(geojson.Feature(geometry=geojson.LineString(path, properties=props)))
        if rebalanced(delta_df):
            feature_list.append(get_rebalancing_geojson_repr(delta_df))
    return geojson.FeatureCollection(feature_list, properties={'bike_id': bike_id})


#     for row in df[df['bikeid'] == bike_id].iterrows():
#         trip = row[1]
#         path = get_bike_path([trip['start station latitude'], trip['start station longitude']],
#                              [trip['end station latitude'], trip['end station longitude']],
#                              client)
#         feature = geojson.Feature(geometry=geojson.LineString(path, properties=trip.to_dict()))
#         feature_list.append(feature)
#     return geojson.FeatureCollection(feature_list, properties={'bike_id': bike_id})

For now I'm working with only a single sample. Once I can verify that this sample works and get the frontend working around it, I can extend the method above to calling it on every bike in our sample dataset and extend the visualization around all of the samples.

In [16]:
# random.choice(list(sample_trips['bikeid'].unique()))
# We got back bikeid no. 20249, which is the bike that we will use!

This bike had 20 trips and 1 rebalancing. Manual inspection confirms that this is the case&mdash;our algorithm is good to go!

In [17]:
one_feature_collection = get_trips(sample_trips, 20249, gmaps)

In [18]:
# one_feature_collection # Too long!

In [19]:
with open('../data/part_1/one_trip.geojson', 'w') as outfile:
    outfile.write(geojson.dumps(one_feature_collection))

## Backend So Far

At this point we've taken the dataset, understood its structure, and generated a single trip's GeoJSON from the result.

What we've done essentially is scoped out and proof-of-concepted the this system's backend. But in order to truly get to the target result here&mdash;a web application served on, well, the web&mdash;a way of operationalizing this backend must be found.

1. A recurrent `chron` job on `PythonAnywhere` will run a script on my website at the beginning of every week which retrieves the files for CitiBike rides from the previous week (stitching together two months' data if necessary), picks ten active bikes, runs the algorithms above on them, gets a GeoJSON repr...
2. and throws them into a `MySQL` data store, also on my `PythonAnywhere` website (this segment is the focus of `A Week in the Life of a CitiBike—Data Store Scoping`).
3. When the visualization page is hit, it will send a request to an internal API, which will process and return a random GeoJSON selection from the `MySQL` data store.
4. From there the front-end kicks in.

For the intro screen I will also statically convert a selection of some fairly large number of random CitiBike paths and plop them onto the map.